# MLS-7

### Connect to the workspace

First, we'll need to connect to your Azure Machine Learning workspace. The Azure Machine Learning workspace is the top-level resource for the service. It provides you with a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We're using DefaultAzureCredential to get access to the workspace. This credential should be capable of handling most Azure SDK authentication scenarios.

In [111]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

from azure.ai.ml import command, Input

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration
)

from azure.storage.blob import BlobServiceClient

Next, get a handle to the workspace by providing your Subscription ID, Resource Group name, and workspace name. To find these parameters:

* Look in the upper-right corner of the Azure Machine Learning studio toolbar for your workspace name.
* Select your workspace name to show your Resource Group and Subscription ID.
* Copy the values for Resource Group and Subscription ID into the code.

In [112]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="1c6d9ef7-1867-436f-986f-f37c475a295b", #Provide your subscription ID as shown in the above screenshot
    resource_group_name="testml", #Provide your Resource Group as shown in the above screenshot
    workspace_name="azureml",
)

### Create a compute resource to run the job

Azure Machine Learning needs a compute resource to run a job. This resource can be single or multi-node machines with Linux or Windows OS, or a specific compute fabric like Spark.

We only need a basic cluster for this task; thus, we'll pick a Standard_D2_v3 model with 2 CPU cores and 7 GB RAM to create an Azure Machine Learning compute.



In [113]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_D2_V3",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=1,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

You already have a cluster named cpu-cluster, we'll reuse it as is.
AMLCompute with name cpu-cluster is created, the compute size is STANDARD_D2_V3


### Configure and submit your training job

#### Prepare the training script

In [114]:
# To use the training script, first create a directory where you will store the file.
import os

src_dir = "./src"
os.makedirs(src_dir, exist_ok=True)

**Next, create the script file in the source directory**

In [115]:
%%writefile {src_dir}/main.py

import os
import mlflow
import argparse

import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingRegressor

def main():

    mlflow.start_run() # Start an MLflow run

    parser = argparse.ArgumentParser() # Create an argument parser
    parser.add_argument("--data", type=str)  # Add an argument for the data file path
    parser.add_argument("--learning-rate", required=False, default=0.1, type=float)
    parser.add_argument("--max-depth", required=False, default=4, type=float)
    parser.add_argument("--n-estimators", required=False, default=100, type=int)
    args = parser.parse_args()

    airbnb = pd.read_csv(args.data)

    from sklearn.model_selection import train_test_split
    X = airbnb.drop(columns=["price"])
    X = pd.get_dummies(X,drop_first=True)

    y = airbnb["price"]

    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
    )
    # Create a Gradient Boosting Regressor model with the specified hyperparameters
    model_gbr = GradientBoostingRegressor(
        n_estimators=args.n_estimators,
        learning_rate=args.learning_rate,
        max_depth=args.max_depth
    )


    model_gbr.fit(X_train, y_train)

    rmse = model_gbr.score(X_test, y_test)

    mlflow.log_metric("RMSE", float(rmse))

    print("Registering model pipeline")
     # Log the trained model to MLflow
    mlflow.sklearn.log_model(
        sk_model=model_gbr,
        registered_model_name="gbr_airbnb_redictor",
        artifact_path="gbr_airbnb_predictor"
    )
    # End the MLflow run

    mlflow.end_run()


if __name__ == '__main__':
    main()



Writing ./src/main.py


**The inputs parameter specifies the input data **



In [116]:
# Import the necessary modules
from azure.ai.ml import command
from azure.ai.ml import Input

# Define a new AML job using the `command` function
job = command(
    inputs=dict(
        data=Input(
            type="uri_file",
            path="azureml:airbnb_data:1", # The path to the input data file
        )),
    # Specify the directory containing the code to be run in the job
    code="./src/",
    # Specify the command to be run in the job, including the input data and parameters as command line arguments
    command="python main.py --data ${{inputs.data}}",
    # Specify the environment to be used for the job
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    # Specify the compute target to be used for the job
    compute="cpu-cluster",
    # Specify the name of the experiment for the job
    experiment_name="airbnb_exper_test",
     # Specify the display name for the job
    display_name="airbnb_display_name",
)


#### Submit the job

In [117]:
# ml_client.create_or_update will create a new job if it does not exist or update the existing job if it does
ml_client.create_or_update(job)

Experiment,Name,Type,Status,Details Page
airbnb_exper_test,sweet_jelly_lwksf11m25,command,Starting,Link to Azure Machine Learning studio


**Note 1: We can check the status and outcome of the training job by going to the "Jobs" on the left --> Metrics.**

**Note: What happens during job execution: **

As the job is executed, it goes through the following stages:

* **Preparing**: A docker image is created according to the environment defined. The image is uploaded to the workspace's container registry and cached for later runs. Logs are also streamed to the run history and can be viewed to monitor progress. If a curated environment is specified, the cached image backing that curated environment will be used.

* **Scaling**: The cluster attempts to scale up if the cluster requires more nodes to execute the run than are currently available.

* **Running**: All scripts in the script folder src are uploaded to the compute target, data stores are mounted or copied, and the script is executed. Outputs from stdout and the ./logs folder are streamed to the run history and can be used to monitor the run.



### Find and register the model

In [118]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes, InputOutputModes

model_name = "gbr_airbnb_redictor_v2"

model = Model(
    name=model_name,
    #The name of the MLflow model.
    path="gbr_airbnb_redictor/gbr_airbnb_predictor",
    #Path to the root directory of the model.
    type=AssetTypes.MLFLOW_MODEL,
    #The type of the model asset(MLflow model).
    description="MLflow model for the airbnb problem",
    #The purpose of the model.
)

We can then register this model.

In [119]:
registered_model = ml_client.models.create_or_update(model=model)

Uploading gbr_airbnb_predictor (0.22 MBs): 100%|██████████| 221411/221411 [00:00<00:00, 1651976.23it/s]




#### Import the required libraries

In [120]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes

#### Create Online Endpoint

Online endpoints are endpoints that are used for online (real-time) inferencing. Online endpoints contain deployments that are ready to receive data from clients and can send responses back in real time.

To create an online endpoint we will use `ManagedOnlineEndpoint`. This class allows user to configure the following key aspects such as `name`,`auth_mode`,`identity`,etc.


#### Configure the endpoint

In [121]:
# Importing the required modules
import random
import string

# Creating a unique endpoint name by including a random suffix

# Defining a list of allowed characters for the endpoint suffix
allowed_chars = string.ascii_lowercase + string.digits

# Generating a random 5-character suffix for the endpoint name by choosing
# characters randomly from the list of allowed characters
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))

# Creating the final endpoint name by concatenating a prefix string
# with the generated suffix string
endpoint_name = "airbnb-endpoint-" + endpoint_suffix


In [122]:
print(f"Endpoint name: {endpoint_name}")

Endpoint name: airbnb-endpoint-4un1w


In [123]:
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,  
    # Name of the endpoint, should be unique within your deployment
    description="An online endpoint serving an MLflow model for the pima classification task",
    # A string describing the purpose of the endpoint
    auth_mode="key",
    # Authentication mode to use for the endpoint (in this case, using an API key)
    tags={"foo": "bar"},
    # A dictionary of key-value pairs that can be used to tag the endpoint
)

#### Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [124]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://airbnb-endpoint-4un1w.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://airbnb-endpoint-4un1w.eastus.inference.ml.azure.com/swagger.json', 'name': 'airbnb-endpoint-4un1w', 'description': 'An online endpoint serving an MLflow model for the pima classification task', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/1c6d9ef7-1867-436f-986f-f37c475a295b/resourcegroups/testml/providers/microsoft.machinelearningservices/workspaces/azureml/onlineendpoints/airbnb-endpoint-4un1w', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/1c6d9ef7-1867-436f-986f-f37c475a295b/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:747c057e-f2cb-42a6-8a0b-069e8e89448d:f37b41ad-5633-434e-b9ea-5778d70117b6?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/1c6d9ef7-1867-4

#### Create a blue deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class. This class allows user to configure key aspects.

##### Curating the deployment script:

**Go to the Microsoft Azure home page and search for Application Insights --> Copy the Connection String and use it in the score.py script.**

In [125]:
%%writefile {src_dir}/score.py

# Import necessary libraries and modules
import logging
import os
import json
import mlflow
from io import StringIO
from mlflow.pyfunc.scoring_server import infer_and_parse_json_input, predictions_to_json

######################LOGGER#####################
# Set up Azure logging
import logging
from logging import Logger
from opencensus.ext.azure.log_exporter import AzureLogHandler

# Connect to Application Insights and set logging level to INFO
application_insights_connection_string= 'InstrumentationKey=e040f1cb-4a7a-4009-87be-7861c813da51;IngestionEndpoint=https://eastus-8.in.applicationinsights.azure.com/;LiveEndpoint=https://eastus.livediagnostics.monitor.azure.com/'
handler = AzureLogHandler(
connection_string=application_insights_connection_string)
logger = logging.getLogger()
logger.addHandler(handler)
logger.setLevel(logging.INFO)

####################################################

# Define the init() function to load the MLflow model
def init():
    global model
    global input_schema
    # "model" is the path of the mlflow artifacts when the model was registered. For automl
    # models, this is generally "mlflow-model".
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "gbr_airbnb_predictor")
    model = mlflow.pyfunc.load_model(model_path)
    input_schema = model.metadata.get_input_schema()

# Define the run() function to make predictions using the loaded model
def run(raw_data):
    # Parse input data
    json_data = json.loads(raw_data)
    if "input_data" not in json_data.keys():
        raise Exception("Request must contain a top level key named 'input_data'")
    serving_input = json.dumps(json_data["input_data"])
    data = infer_and_parse_json_input(serving_input, input_schema)

    # Make predictions
    predictions = model.predict(data)

    # Log the input data and predictions to Azure
    logger.info("Data:{0},Predictions:{1}".format(str(data),str(predictions)))

    # Convert predictions to JSON format and return
    result = StringIO()
    predictions_to_json(predictions, result)
    return result.getvalue()


Writing ./src/score.py


**Curating the deployment:**

In [126]:
# Create a new deployment with name "blue"
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    # Use the previously generated endpoint name
    endpoint_name=endpoint_name,
    # Use the registered model
    model=registered_model,
    # Use the latest environment
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    # Use the code in the "./src" directory and the "score.py" script
    code_configuration=CodeConfiguration(
        code="./src", scoring_script="score.py"
    ),
    # Use a single instance of type "Standard_E2s_v3"
    instance_type="Standard_E2s_v3",
    instance_count=1,
    # Enable Application Insights for the deployment
    app_insights_enabled=True,
)

#### Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [127]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint airbnb-endpoint-4un1w exists


................................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'airbnb-endpoint-4un1w', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/1c6d9ef7-1867-436f-986f-f37c475a295b/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/od:747c057e-f2cb-42a6-8a0b-069e8e89448d:fab0816e-da5f-4713-9488-b16456016a81?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/1c6d9ef7-1867-436f-986f-f37c475a295b/resourceGroups/testml/providers/Microsoft.MachineLearningServices/workspaces/azureml/onlineEndpoints/airbnb-endpoint-4un1w/deployments/blue', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/testlab/code/Users/TESTXRC71JBY4S_1686047967735/w7_monitoring_p1', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fc7fb5a6800

### Directional Expectation

This task involves monitoring the alignment of model predictions with the expected direction of change. In many scenarios, organizations have a predefined understanding of how certain variables should impact the outcome.  Monitoring the directional expectation involves analyzing whether the model's predictions align with these predefined expectations. By comparing the model's predictions with the anticipated direction of change, organizations can quickly identify potential issues or anomalies that require further investigation. This task allows for an initial assessment of the model's performance and can provide insights into any unexpected behaviors or deviations from the expected patterns.

In [129]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
        deployment_name="blue",
        request_file="baseline-data.json"
)

In [130]:
print(response)

"{\"predictions\": [5.02775838674696]}"


In [132]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
        deployment_name="blue",
        request_file="test-data.json"
)

In [133]:
print(response)

"{\"predictions\": [4.917771115184388]}"


### Perturbation Analysis

Perturbation analysis involves introducing deliberate changes or perturbations to the input data and observing the corresponding impact on model predictions. This task helps evaluate the stability and robustness of the model. By systematically perturbing variables or introducing simulated variations, organizations can assess how sensitive the model is to different inputs and determine if it responds in an expected manner. For instance, in a credit risk assessment model, perturbation analysis could involve altering individual features such as income or credit utilization ratios to observe how the model's predictions change. This analysis helps identify potential vulnerabilities or inconsistencies in the model's behavior and informs the need for recalibration or retraining.

In [136]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
        deployment_name="blue",
        request_file="baseline-payload-data.json"
)

In [137]:
print(response)

"{\"predictions\": [4.988102857005535]}"


In [138]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
        deployment_name="blue",
        request_file="test-payload-data.json"
)

In [139]:
print(response)

"{\"predictions\": [5.037330794542146]}"


### Monitoring Critical Subgroups

In some applications, it is important to monitor the performance of machine learning models specifically for critical subgroups or segments of the population. These subgroups may be defined by demographic characteristics, geographic location, or other relevant factors. For example, in healthcare, it is crucial to ensure that a medical diagnosis model performs well across different demographic groups to avoid bias or disparities in patient care. Short-term monitoring tasks involve regularly assessing model performance metrics, such as accuracy or false positive rates, specifically for these critical subgroups. If significant disparities or performance gaps are detected, further investigation can be conducted to understand the root causes and take necessary corrective actions, such as retraining the model or adjusting its decision thresholds.

In [143]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
        deployment_name="blue",
        request_file="critical-sku-payload-data.json"
)

In [144]:
print(response)

"{\"predictions\": [6.1218026828745975]}"
